In [228]:
import pandas as pd

# 파일 경로
train_path = "/Users/eunzinri/Downloads/next-day-air-temperature-forecast-challenge/train_dataset.csv"
station_path = "/Users/eunzinri/Downloads/next-day-air-temperature-forecast-challenge/station_info.csv"

# CSV 파일 불러오기
train_df = pd.read_csv(train_path)
station_df = pd.read_csv(station_path)

# 데이터프레임 크기와 간단한 정보 출력
train_df.info(), station_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13132 entries, 0 to 13131
Columns: 342 entries, id to target
dtypes: float64(338), int64(2), object(2)
memory usage: 34.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   지점                9 non-null      int64  
 1   시작일               9 non-null      object 
 2   종료일               1 non-null      object 
 3   지점명               9 non-null      object 
 4   위도                9 non-null      float64
 5   경도                9 non-null      float64
 6   노장해발고도(m)         9 non-null      float64
 7   기압계(관측장비지상높이(m))  9 non-null      float64
 8   기온계(관측장비지상높이(m))  9 non-null      float64
 9   풍속계(관측장비지상높이(m))  9 non-null      float64
 10  강우계(관측장비지상높이(m))  9 non-null      float64
dtypes: float64(7), int64(1), object(3)
memory usage: 924.0+ bytes


(None, None)

In [230]:
import numpy as np

def preprocess_data(train_df, station_df):
    df = train_df.copy()
    
    # 1. -9999를 NaN으로 변환
    df.replace(-9999, np.nan, inplace=True)

    # 2. 날짜 분리 (month, day)
    df[['month', 'day']] = df['date'].str.split('-', expand=True).astype(int)
    df.drop(columns=['date'], inplace=True)

    # 3. 결측치 처리
    # - 자연적인 결측치로 보이는 건 0으로
    for col in df.columns:
        if 'sunshine_duration' in col or 'snow_depth' in col:
            df[col].fillna(0, inplace=True)
        if "min_cloud_height" in col:
            df[col].fillna(90, inplace=True)
    
    # - 나머지는 평균값으로 (또는 향후 보간 가능)
    df.fillna(df.mean(numeric_only=True), inplace=True)

    # 4. 관측소 정보 병합
    station_df_renamed = station_df.rename(columns={
        '지점': 'station',
        '위도': 'latitude',
        '경도': 'longitude',
        '노장해발고도(m)': 'altitude'
    })[['station', 'latitude', 'longitude', 'altitude']]
    
    station_df_renamed = station_df_renamed.drop_duplicates(subset='station')
    
    
    df = df.merge(station_df_renamed, on='station', how='left')

    # 6. 불필요 컬럼 제거
    df.drop(columns=['station_name'], inplace=True)
    df.drop(columns=['station'], inplace=True)
    df.drop(columns=['id'], inplace=True)

    return df

# 전처리 적용
processed_df = preprocess_data(train_df, station_df)
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13132 entries, 0 to 13131
Columns: 343 entries, cloud_cover_0 to altitude
dtypes: float64(341), int64(2)
memory usage: 34.4 MB


/var/folders/d2/9rzglf1n68j37h1zk5c5zvkw0000gn/T/ipykernel_35232/2249579865.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(90, inplace=True)
/var/folders/d2/9rzglf1n68j37h1zk5c5zvkw0000gn/T/ipykernel_35232/2249579865.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as 

In [232]:
processed_df.head

<bound method NDFrame.head of        cloud_cover_0  cloud_cover_1  cloud_cover_10  cloud_cover_11  \
0                0.0            0.0             9.0             0.0   
1                0.0            0.0             0.0             0.0   
2                0.0            0.0             0.0             0.0   
3                0.0            0.0             2.0             0.0   
4                0.0            0.0             0.0             0.0   
...              ...            ...             ...             ...   
13127            2.0            4.0             0.0             0.0   
13128            0.0            0.0             0.0             0.0   
13129            4.0            1.0             0.0             0.0   
13130            9.0            9.0             0.0             0.0   
13131            6.0            3.0             1.0             2.0   

       cloud_cover_12  cloud_cover_13  cloud_cover_14  cloud_cover_15  \
0                 3.0             3.0       

In [234]:
processed_df.dtypes

cloud_cover_0     float64
cloud_cover_1     float64
cloud_cover_10    float64
cloud_cover_11    float64
cloud_cover_12    float64
                   ...   
month               int64
day                 int64
latitude          float64
longitude         float64
altitude          float64
Length: 343, dtype: object

In [186]:
!pip install xgboost

In [236]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

# 전처리된 데이터 불러온 후 float32로 변환
df = processed_df

X = df.drop(columns=['target'])
y = df['target']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = XGBRegressor(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=6,
    random_state=42,
    tree_method='hist'
)

model.fit(X_train, y_train)
y_pred = model.predict(X_val)

rmse = mean_squared_error(y_val, y_pred, squared=False)
print(f'✅ 검증 RMSE: {rmse:.4f} °C')

✅ 검증 RMSE: 1.4943 °C


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [238]:
model = XGBRegressor(
    n_estimators=300,
    learning_rate=0.03,
    max_depth=8,
    random_state=42,
    tree_method='exact'
)

model.fit(X_train, y_train)
y_pred = model.predict(X_val)

rmse = mean_squared_error(y_val, y_pred, squared=False)
print(f'✅ 검증 RMSE: {rmse:.4f} °C')

✅ 검증 RMSE: 1.4512 °C


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [56]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import numpy as np
from xgboost import XGBRegressor

def run_kfold_xgb(X, y, n_splits=10):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    rmses = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        print(f"🌱 Fold {fold + 1}")
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model = XGBRegressor(
            n_estimators=300,
            learning_rate=0.03,
            max_depth=8,
            random_state=42,
            tree_method="hist"
        )
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        rmse = mean_squared_error(y_val, preds, squared=False)
        print(f"  📏 RMSE: {rmse:.4f}")
        rmses.append(rmse)

    print(f"\n✅ 평균 RMSE: {np.mean(rmses):.4f} ± {np.std(rmses):.4f}")
    return rmses

run_kfold_xgb(X, y)


🌱 Fold 1


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


  📏 RMSE: 1.4307
🌱 Fold 2


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


  📏 RMSE: 1.4608
🌱 Fold 3


...

  📏 RMSE: 1.4281
🌱 Fold 9


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


  📏 RMSE: 1.3588
🌱 Fold 10
  📏 RMSE: 1.4092

✅ 평균 RMSE: 1.4306 ± 0.0300


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


[1.4306703259824431,
 1.4607670595603344,
 1.4782299193393587,
 1.430660599042766,
 1.4427151492529406,
 1.4286525903657676,
 1.4383742295955755,
 1.4280609553825783,
 1.358849479984743,
 1.4092150595239676]

In [240]:
final_model = XGBRegressor(
    n_estimators=300,
    learning_rate=0.03,
    max_depth=8,
    random_state=42,
    tree_method="exact"
)

# 💥 전부 다 학습에 씀
final_model.fit(X, y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.03, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=300,
             n_jobs=None, num_parallel_tree=None, ...)

In [242]:
test_path = "/Users/eunzinri/Downloads/next-day-air-temperature-forecast-challenge/test_dataset.csv"
test_df = pd.read_csv(test_path)

In [244]:
test_df.count

<bound method DataFrame.count of         id  station station_name   date  cloud_cover_0  cloud_cover_1  \
0        0       99           파주  06-26            0.0            0.0   
1        1      119           수원  01-11            0.0            0.0   
2        2      119           수원  02-23            5.0            6.0   
3        3      119           수원  05-29            0.0            0.0   
4        4      119           수원  06-15            0.0            4.0   
...    ...      ...          ...    ...            ...            ...   
4374  4374      119           수원  06-10            1.0            5.0   
4375  4375       99           파주  04-12            0.0            0.0   
4376  4376      119           수원  06-23            4.0            5.0   
4377  4377      119           수원  05-04            4.0            5.0   
4378  4378       99           파주  05-11            0.0            0.0   

      cloud_cover_10  cloud_cover_11  cloud_cover_12  cloud_cover_13  ...  \
0            

In [246]:
test = preprocess_data(test_df, station_df)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4379 entries, 0 to 4378
Columns: 342 entries, cloud_cover_0 to altitude
dtypes: float64(340), int64(2)
memory usage: 11.4 MB


/var/folders/d2/9rzglf1n68j37h1zk5c5zvkw0000gn/T/ipykernel_35232/2249579865.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(90, inplace=True)
/var/folders/d2/9rzglf1n68j37h1zk5c5zvkw0000gn/T/ipykernel_35232/2249579865.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as 

In [248]:
test.count

<bound method DataFrame.count of       cloud_cover_0  cloud_cover_1  cloud_cover_10  cloud_cover_11  \
0               0.0            0.0             0.0             1.0   
1               0.0            0.0             0.0             0.0   
2               5.0            6.0             0.0             0.0   
3               0.0            0.0             0.0             0.0   
4               0.0            4.0             2.0             4.0   
...             ...            ...             ...             ...   
4374            1.0            5.0             1.0             2.0   
4375            0.0            0.0             0.0             1.0   
4376            4.0            5.0             8.0             2.0   
4377            4.0            5.0             9.0             7.0   
4378            0.0            0.0             0.0             0.0   

      cloud_cover_12  cloud_cover_13  cloud_cover_14  cloud_cover_15  \
0                3.0             6.0             9.0  

In [250]:
prediction = final_model.predict(test)

In [252]:
submission_df = pd.DataFrame({'target':prediction})

In [254]:
submission_df.to_csv('newjeans.csv',index=True)